<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Описание-данных" data-toc-modified-id="Описание-данных-0.1"><span class="toc-item-num">0.1&nbsp;&nbsp;</span>Описание данных</a></span></li><li><span><a href="#Условия-задачи:" data-toc-modified-id="Условия-задачи:-0.2"><span class="toc-item-num">0.2&nbsp;&nbsp;</span>Условия задачи:</a></span></li></ul></li><li><span><a href="#Загрузка-и-подготовка-данных" data-toc-modified-id="Загрузка-и-подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка и подготовка данных</a></span><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Подготовка данных</a></span></li></ul></li><li><span><a href="#Обучение-и-проверка-модели" data-toc-modified-id="Обучение-и-проверка-модели-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение и проверка модели</a></span></li><li><span><a href="#Подготовка-к-расчёту-прибыли" data-toc-modified-id="Подготовка-к-расчёту-прибыли-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Подготовка к расчёту прибыли</a></span></li><li><span><a href="#Расчёт-прибыли-и-рисков" data-toc-modified-id="Расчёт-прибыли-и-рисков-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Расчёт прибыли и рисков</a></span><ul class="toc-item"><li><span><a href="#Функция-для-расчёта-прибыли" data-toc-modified-id="Функция-для-расчёта-прибыли-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Функция для расчёта прибыли</a></span></li><li><span><a href="#Bootstrap" data-toc-modified-id="Bootstrap-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Bootstrap</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Чек-лист-готовности-проекта" data-toc-modified-id="Чек-лист-готовности-проекта-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист готовности проекта</a></span></li></ul></div>

# Выбор локации для скважины

Нужно решить, где бурить новую скважину.

Нам предоставлены пробы нефти в трёх регионах: в каждом 100 000 месторождений, где измерили качество нефти и объём её запасов. Построим модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

### Описание данных
Данные геологоразведки трёх регионов находятся в файлах:
- /datasets/geo_data_0.csv
- /datasets/geo_data_1.csv
- /datasets/geo_data_2.csv

id — уникальный идентификатор скважины;
<br>f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы);
<br>product — объём запасов в скважине (тыс. баррелей).

### Условия задачи:
* Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).
* При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
* Бюджет на разработку скважин в регионе — 10 млрд рублей.
* При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.
* После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.

Данные синтетические: детали контрактов и характеристики месторождений не разглашаются.

## Загрузка и подготовка данных

### Загрузка данных

In [1]:
import pandas as pd
import datetime as dt
import numpy as np
import os
import sklearn as sk
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
#from sklearn.experimental import enable_halving_search_cv 
#from sklearn.model_selection import HalvingGridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.utils import shuffle
import warnings
#from random import seed, random
#import statistics

In [2]:
dt.datetime.today().strftime("%d.%m.%Y %H:%M")

'22.06.2022 17:17'

In [3]:
warnings.filterwarnings("ignore", category=DeprecationWarning)
# warnings.filterwarnings("ignore", category=FutureWarning)

In [4]:
df = []
pth1 = '/datasets/'
pth2 = ''
if os.path.exists(pth1+'geo_data_0.csv'):
    df.append(pd.read_csv(pth1+'geo_data_0.csv'))
    df.append(pd.read_csv(pth1+'geo_data_1.csv'))
    df.append(pd.read_csv(pth1+'geo_data_2.csv'))
elif os.path.exists(pth2+'geo_data_0.csv'):
    df.append(pd.read_csv(pth2+'geo_data_0.csv'))
    df.append(pd.read_csv(pth2+'geo_data_1.csv'))
    df.append(pd.read_csv(pth2+'geo_data_2.csv'))
else:
    print('Something is wrong')

In [5]:
df[0].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [6]:
df[0].head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


Из описания данных:

`id` — уникальный идентификатор скважины;<br>
`f0`, `f1`, `f2` — три признака точек (неважно, что они означают, но сами признаки значимы);<br>
`product` — объём запасов в скважине (тыс. баррелей).

In [7]:
df[1].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [8]:
df[2].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


### Подготовка данных

Предобработка данных в этом проекте не потребуется, т.к. признаки все признаки без пропусков, числовые, нормализованные (и вообще синтетические).

Выделим целевой признак.

In [9]:
target = []
features = []
for i in range(3):
    target.append(df[i]['product'])
    features.append(df[i][['f0','f1','f2']])

In [10]:
target[0].head(2)

0    105.280062
1     73.037750
Name: product, dtype: float64

In [11]:
features[0].head(2)

,f0,f1,f2
0,0.705745,-0.497823,1.22117
1,1.334711,-0.340164,4.36508


Разобьём данные на обучающую и валидационную выборки в соотношении 75:25.

In [12]:
features_train = []
features_valid = []
target_train = []
target_valid = []
for i in range(3):
    features_train.append(pd.Series())
    features_valid.append(pd.Series())
    target_train.append(pd.Series())
    target_valid.append(pd.Series())
    features_train[i], features_valid[i], target_train[i], target_valid[i] = train_test_split(
        features[i], target[i], 
        test_size=0.25, 
        shuffle=True,
        random_state=486745)
    target_valid[i].reset_index(drop=True, inplace=True)
    features_valid[i].reset_index(drop=True, inplace=True)

print(features_train[0].shape) 
print(features_valid[0].shape) 
for i in range(3):
    print( target_train[i].mean(), target_valid[i].mean(), sep='\t')

(75000, 3)
(25000, 3)
92.58611961340254	92.24164115979244
68.73114736286965	69.10655791139102
95.08637922058705	94.7408623382389


## Обучение и проверка модели

Обучим модель и сделаем предсказания на валидационной выборке. Сохраним предсказания и правильные ответы на валидационной выборке.

\* *Не удалось использовать HalvingGridSearchCV из-за версии scikit-learn окружения Praktikum.*

In [13]:
%%time

params = dict(fit_intercept=[True,False]
              , normalize=[True,False]
             )

model = []
predicted_valid = []

for i in range(3):
    model.append(LinearRegression())
    grid = GridSearchCV(model[i], params, verbose=0, scoring='neg_mean_squared_error', cv=5)
    grid.fit(features_train[i], target_train[i])
    
    print(i+1, '-й регион:', sep='')
    print(grid.best_params_)
    print('score:', grid.best_score_)
    model[i] = grid.best_estimator_
    print(model[i])
    
    predicted_valid.append(pd.Series(model[i].predict(features_valid[i])))
    print('Средний запас предсказанного сырья на валидационной выборке:',
          predicted_valid[i].mean(),
          'тыс. баррелей')
    print('Средний фактический запас:',
          target_valid[i].mean(),
          'тыс. баррелей')
    print('RMSE на валидационной выборке:',
          mean_squared_error(target_valid[i], predicted_valid[i]
                             #, squared=False
                            ) ** 0.5,
          '\n'
         )

1-й регион:
{'fit_intercept': True, 'normalize': True}
score: -1416.1829051387851
LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=True)
Средний запас предсказанного сырья на валидационной выборке: 92.60236164020787 тыс. баррелей
Средний фактический запас: 92.24164115979244 тыс. баррелей
RMSE на валидационной выборке: 37.878920074603954 

2-й регион:
{'fit_intercept': True, 'normalize': False}
score: -0.7895016786087907
LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)
Средний запас предсказанного сырья на валидационной выборке: 69.10352620076132 тыс. баррелей
Средний фактический запас: 69.10655791139102 тыс. баррелей
RMSE на валидационной выборке: 0.8960369341577493 

3-й регион:
{'fit_intercept': True, 'normalize': True}
score: -1605.630552184123
LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=True)
Средний запас предсказанного сырья на валидационной выборке: 94.93003149492975 тыс. баррелей
Средний фактиче

2-й регион сильно отличается от 1-го и  3-го: для него меньше предсказанный средний запас, но на 2 порядка ниже среднеквадратичная ошибка.

## Подготовка к расчёту прибыли

Все ключевые значения для расчётов сохраним в отдельных переменных.

In [14]:
N_EXPLORING = 500  # исследуют 500 скважин региона
N_BEST = 200  # из них выбирают 200 лучших для разработки

UNIT_INCOME = 450_000  # (руб.) - доход с единицы продукта (тыс. баррелей)

TOTAL_BUDGET = 10_000_000_000  # (руб.) - бюджет на разработку (200?) скважин в регионе
budget = TOTAL_BUDGET / N_BEST  # бюджет на разработку 1 скважины
print('budget =', budget)

MIN_LOSS_PROBABILITY = 0.025  # После оценки рисков нужно оставить лишь те регионы,
                              # в которых вероятность убытков меньше 2.5%.

budget = 50000000.0


Рассчитаем достаточный объём сырья для безубыточной разработки новой скважины. Сравним полученный объём сырья со средним запасом в каждом регионе.

In [15]:
enough_volume = budget / UNIT_INCOME
enough_volume

111.11111111111111

Во 2-м регионе предсказанный средний запас только ~69 тыс. баррелей, что гораздо ниже достаточного объёма для безубыточной разработки. В 1-м и 3-м тоже ниже, но не так существенно.

## Расчёт прибыли и рисков 

### Функция для расчёта прибыли

Напишем функцию для расчёта прибыли по выбранным скважинам и предсказаниям модели:
- Выберем скважины с максимальными значениями предсказаний.
- Просуммируем целевое значение объёма сырья, соответствующее этим предсказаниям.
- Рассчитаем прибыль для полученного объёма сырья.

In [16]:
def profit_calc (predicted, target):
    chosen_predicted = predicted.sort_values(ascending=False)[:N_BEST]
    chosen_target = target[chosen_predicted.index]
#     print(chosen_predicted.head(3))
#     print(chosen_target.head(3))
    
    profit = chosen_target.sum() * UNIT_INCOME - TOTAL_BUDGET
    return profit

Проверим работу функции:

In [17]:
predicted_sample = predicted_valid[0].sample(n=N_EXPLORING,replace=True,random_state=54982)
#target_sample = target_valid[0].loc[predicted_sample.index]
profit = profit_calc(predicted_sample, target_valid[0])
print(f"прибыль: {profit:10,.2f}")

прибыль: 5,202,841.56


### Bootstrap

Посчитаем риски и прибыль для каждого региона:
- Применим технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли.
- Найдём среднюю прибыль, 95%-й доверительный интервал и риск убытков. Убыток — это отрицательная прибыль.
- Сделаем выводы: предложим регион для разработки скважин и обоснуем выбор.

In [19]:
%%time
state = np.random.RandomState(12345)
values = []

for i in range(3):
    print(i+1, '-й регион:', sep='')
    values.append([])
    
    for j in range(1000):
        predicted_sample = predicted_valid[i].sample(n=N_EXPLORING, replace=True, random_state=state)
        #target_sample = target_valid[i].loc[predicted_sample.index]
        values[i].append(profit_calc(predicted_sample, target_valid[i]))
    
    values[i] = pd.Series(values[i])
    mean = values[i].mean()
    q_0025 = values[i].quantile(q=0.025)
    q_0975 = values[i].quantile(q=0.975)
    risk_of_loss = values[i].loc[values[i] < 0].count() / values[i].count()
    
    print(f"Средняя прибыль: {mean:12,.2f}")
    print(f"Доверительный интервал 95% для средней прибыли: [{q_0025:12,.2f};  {q_0975:12,.2f}]")
    print(f"Риск убытков (отриц.прибыль): {risk_of_loss*100 :3.1f}%")
    #print(values[i].describe())
    print('\n')

1-й регион:
Средняя прибыль: 377,798,930.43
Доверительный интервал 95% для средней прибыли: [-110,964,377.04;  883,580,483.13]
Риск убытков (отриц.прибыль): 6.5%


2-й регион:
Средняя прибыль: 466,393,727.43
Доверительный интервал 95% для средней прибыли: [69,451,683.82;  860,831,045.95]
Риск убытков (отриц.прибыль): 0.9%


3-й регион:
Средняя прибыль: 349,643,697.10
Доверительный интервал 95% для средней прибыли: [-170,562,982.98;  882,175,655.01]
Риск убытков (отриц.прибыль): 7.7%


Wall time: 2.7 s


### Вывод
И 2-й, и 3-й регион сразу исключаем из-за риска убытков >2.5%.
<br>Для 2-го региона выше и средняя прибыль, и нижняя граница доверительного интервала 95% средней прибыли, и ниже риск убытков. Ниже только верхняя граница доверительного интервала, но не намного. Это однозначный победитель, несмотря на меньший средний запас. Меньшая среднеквадратичная ошибка и более высокая предсказуемость данных в этом регионе в итоге сыграла решающую роль.

<div class="alert alert-success">
<b> ✔️ Комментарий ревьювера:</b> Вывод правильный. Действительно, регион 2 вытянул по безубыточности за счет высокого качества предсказания.
    
Однако если посмотреть на матрицу корреляции, можно увидеть странную высокую корреляцию одного из признаков и целевой переменной. Возможно в данных ошибка, но в данной работе мы это не рассматриваем.
    
Вот небольшая статья по бутстрапу в том числе, вдруг заинтересует:
    
https://habr.com/ru/company/ods/blog/324402/
</div>

In [20]:
dt.datetime.today().strftime("%d.%m.%Y %H:%M")

'22.06.2022 17:17'

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные подготовлены
- [x]  Выполнен шаг 2: модели обучены и проверены
    - [x]  Данные корректно разбиты на обучающую и валидационную выборки
    - [x]  Модели обучены, предсказания сделаны
    - [x]  Предсказания и правильные ответы на валидационной выборке сохранены
    - [x]  На экране напечатаны результаты
    - [x]  Сделаны выводы
- [x]  Выполнен шаг 3: проведена подготовка к расчёту прибыли
    - [x]  Для всех ключевых значений созданы константы Python
    - [x]  Посчитано минимальное среднее количество продукта в месторождениях региона, достаточное для разработки
    - [x]  По предыдущему пункту сделаны выводы
    - [x]  Написана функция расчёта прибыли
- [x]  Выполнен шаг 4: посчитаны риски и прибыль
    - [x]  Проведена процедура *Bootstrap*
    - [x]  Все параметры бутстрепа соответствуют условию
    - [x]  Найдены все нужные величины
    - [x]  Предложен регион для разработки месторождения
    - [x]  Выбор региона обоснован